In [6]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.cross_validation import KFold

import revrand.basis_functions as bf
from revrand.validation import smse, msll
from revrand.regression import learn, predict

In [7]:
# Load the data
boston = load_boston()
X = boston.data
y = boston.target - boston.target.mean()

N, D = X.shape

In [8]:
# Construct basis functions
base = bf.RandomRBF_ARD(Xdim=D, nbases=50) # + bf.LinearBasis(onescol=True)
init_params = [1000. * np.ones(D)]

In [9]:
# Cross val
folds = 5
av_smse = 0.
av_msll = 0.
for i, (tr_ind, ts_ind) in enumerate(KFold(len(y), n_folds=folds, shuffle=True)):
    
    # Training
    params = learn(X[tr_ind], y[tr_ind], base, init_params)
    
    # Prediction
    Ey, Vf, Vy = predict(X[ts_ind], base, *params)
    
    # Validation
    f_smse = smse(y[ts_ind], Ey)
    f_msll = msll(y[ts_ind], Ey, Vy, y[tr_ind])
    av_smse += f_smse
    av_msll += f_msll
    
    print("Fold: {}, SMSE = {}, MSLL = {}".format(i, f_smse, f_msll))
    
av_smse /= folds
av_msll /= folds

Fold: 0, SMSE = 0.15466592634805312, MSLL = -0.8722330608320378
Fold: 1, SMSE = 0.11891174029198191, MSLL = -1.098062491853892
Fold: 2, SMSE = 0.11031567631125858, MSLL = -1.1260108980426107
Fold: 3, SMSE = 0.31891256504588256, MSLL = -0.6093626106878968
Fold: 4, SMSE = 0.07985445960530245, MSLL = -1.283144312784024


In [10]:
# Print results
print("Final: SMSE = {}, MSLL = {}".format(av_smse, av_msll))

Final: SMSE = 0.15653207352049575, MSLL = -0.9977626748400923
